In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

### 구글 메일 발송 Tools

In [ ]:
# from langchain_google_community import GmailToolkit
# from langchain_google_community.gmail.utils import (
#     build_resource_service,
#     get_gmail_credentials,
# )

# # Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# # For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
# credentials = get_gmail_credentials(
#     token_file="./google/token.json",
#     scopes=["https://mail.google.com/"],
#     client_secrets_file="./google/credentials.json",
# )
# api_resource = build_resource_service(credentials=credentials)
# gmail_toolkit = GmailToolkit(api_resource=api_resource)
# gmail_tool_list = gmail_toolkit.get_tools()

### ArXiv Tools

In [ ]:
from langchain.agents import load_tools

loaded_tool_list = load_tools(
    ["arxiv"],
)

#### Retriever Tools

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name = 'real_estate_tax',
    persist_directory = './real_estate_tax_collection'
)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="real_estate_tax_retriever",
    description="Contains information about real estate tax up to December 2024",
)

In [ ]:
from langgraph.prebuilt import ToolNode

tool_list = [add, multiply, search_tool] # + gmail_tool_list
tool_list += loaded_tool_list
tool_list += [retriever_tool]
llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

In [ ]:
# multiply.invoke({"a": 3, "b": 5})

In [ ]:
ai_message = llm_with_tools.invoke("What is 3 plus 5?")
ai_message

In [ ]:
tool_node.invoke({"messages": [ai_message]}) # list[AnyMessage], 마지막 AIMessage, tool_calls를 포함할 것

In [ ]:
from langgraph.graph import MessagesState, StateGraph

# 요약하기 위해 오버라이딩
class AgentState(MessagesState):
    summary: str

graph_builder = StateGraph(AgentState)

#### 메세지 요약

In [ ]:
def summarize_messages(state: AgentState):
    messages = state['messages']
    summary = state['summary']
    summary_prompt = f"summarize this chat history below: \n\nchat_history:{messages}"
    if summary != "":
        summary_prompt += f'''summarize this chat history below while looking at the summary of earlier conversations
chat_history:{messages}
summary:{summary}'''
    summary = small_llm.invoke(summary_prompt)
    response = llm_with_tools.invoke(messages)
    return {'summary': response.content}


In [ ]:
from langchain_core.messages import SystemMessage

def agent(state: AgentState):
    messages = state['messages']
    summary = state['summary']
    if summary != "":
        messages = [SystemMessage(content=f"Here is the summary of the earlier conversation: {summary}")] + messages
    response = llm_with_tools.invoke(messages)
    return {'messages': [response]}

#### Node 방식으로 삭제

In [ ]:
from langchain_core.messages import RemoveMessage

def delete_messages(state: AgentState):
    messages = state['messages']
    delete_messages = [RemoveMessage(id=message.id) for message in messages[:-3]]
    return {'messages': delete_messages}

In [ ]:
from langgraph.graph import END

def should_continue(state: AgentState):
    messages = state['messages']
    last_ai_message = messages[-1]
    if last_ai_message.tool_calls:
        return 'tools'
    # return 'delete_messages'
    return 'summarize_messages'

In [ ]:
graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)
# graph_builder.add_node('delete_messages', delete_messages)
graph_builder.add_node(delete_messages) # node 이름과 함수이름이 같으면 생략가능
graph_builder.add_node(summarize_messages)

In [ ]:
from langgraph.graph import START, END
from langgraph.prebuilt import tools_condition

graph_builder.add_edge(START, 'agent')
# graph_builder.add_conditional_edges(
#     'agent',
#     should_continue,
#     ['tools', END],
# )
## tools_condition를 사용해서 변경
# graph_builder.add_conditional_edges(
#     'agent',
#     tools_condition,
# )
## delete_message를 사용해서 변경
graph_builder.add_conditional_edges(
    'agent',
    should_continue,
    ['tools', 'summarize_messages'],
)
graph_builder.add_edge('tools', 'agent')
graph_builder.add_edge('summarize_messages', 'delete_messages')
graph_builder.add_edge('delete_messages', END)

#### Chat History

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()

In [ ]:
graph = graph_builder.compile(
    checkpointer=checkpointer
)

In [ ]:
# %%capture --no-strerr

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

config = {
    'configurable': {
        'thread_id': 'paper_summary'
    }
}

# query = "What currency is in Billy Giles\' birthplace?"
# query = "Attention Is All You Need라는 논문을 요약해서 설명해줘."
# query = "집이 15억일 때 종합부동산세를 계산해줄 수 있나요?"
query = "Attention Is All You Need라는 논문을 요약해서 이메일 초안 작성해줘."

for chunk in graph.stream({'messages': [HumanMessage(query)], 'summary':''}, config=config, stream_mode='values'):
    chunk['messages'][-1].pretty_print()

In [ ]:
current_message_list = graph.get_state(config).values['messages']
current_message_list

#### 최근 하나의 메시지만 남기고 삭제
##### 토큰 절약 -> 비용 절약, 시간 절약

In [ ]:
# from langchain_core.messages import RemoveMessage

# for index, message in enumerate(current_message_list):
#     if index < len(current_message_list) - 1:
#         graph.update_state(config, {'messages': RemoveMessage(id=message.id)})

In [ ]:
current_message_list = graph.get_state(config).values['messages']
current_message_list

In [ ]:
graph.get_state(config).values['summary']

In [ ]:
# update_query = "논문의 출처 url을 첨부해주세요"
# for chunk in graph.stream({'messages': [HumanMessage(update_query)]}, config=config, stream_mode='values'):
#     chunk['messages'][-1].pretty_print()